In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.model_selection._validation import _backtesting_forecaster
from skforecast.preprocessing import RollingFeatures

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

# Fixtures
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series

In [38]:
forecaster = ForecasterRecursiveMultiSeries(
    regressor          = Ridge(random_state=123),
    lags               = 2,
    transformer_series = None,
    encoding           = "onehot",
)
forecaster.fit(series=series)

cv = TimeSeriesFold(
        initial_train_size = None,
        steps              = 3,
        refit              = False,
        fixed_train_size   = False
        )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster            = forecaster,
                                            series                = series,
                                            cv                    = cv,
                                            levels                = None,
                                            metric                = 'mean_absolute_error',
                                            add_aggregated_metric = False,
                                            exog                  = None,
                                            verbose               = False,
                                            return_predictors=True,
                                        )
backtest_predictions

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:471                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

  0%|          | 0/16 [00:00<?, ?it/s]

,level,pred,fold,lag_1,lag_2,l1,l2
2,l1,0.484591,0,0.286139,0.696469,1.0,0.0
2,l2,0.531310,0,0.826341,0.120629,0.0,1.0
3,l1,0.507604,0,0.484591,0.286139,1.0,0.0
3,l2,0.494425,0,0.531310,0.826341,0.0,1.0
4,l1,0.503402,0,0.507604,0.484591,1.0,0.0
...,...,...,...,...,...,...,...
47,l2,0.476049,15,0.151127,0.691702,0.0,1.0
48,l1,0.503498,15,0.508510,0.483034,1.0,0.0
48,l2,0.510063,15,0.476049,0.151127,0.0,1.0
49,l1,0.502495,15,0.503498,0.508510,1.0,0.0


In [39]:
np.testing.assert_array_almost_equal(
    forecaster.regressor.predict(backtest_predictions[['lag_1', 'lag_2', 'l1', 'l2']]),
    backtest_predictions['pred'].to_numpy(),
)

In [31]:
series_2 = pd.DataFrame({'l1': pd.Series(np.arange(start=0, stop=50, dtype=float)), 
                         'l2': pd.Series(np.arange(start=50, stop=100, dtype=float))})

In [32]:
forecaster = ForecasterRecursiveMultiSeries(
                     regressor          = LinearRegression(),
                     lags               = 5,
                     encoding           = 'ordinal'
                )
forecaster.fit(series=series_2)
results = forecaster.create_predict_X(steps=5, levels=None)
results

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:471                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

,level,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast
50,l1,49.0,48.0,47.0,46.0,45.0,0.0
50,l2,99.0,98.0,97.0,96.0,95.0,1.0
51,l1,50.0,49.0,48.0,47.0,46.0,0.0
51,l2,100.0,99.0,98.0,97.0,96.0,1.0
52,l1,51.0,50.0,49.0,48.0,47.0,0.0
52,l2,101.0,100.0,99.0,98.0,97.0,1.0
53,l1,52.0,51.0,50.0,49.0,48.0,0.0
53,l2,102.0,101.0,100.0,99.0,98.0,1.0
54,l1,53.0,52.0,51.0,50.0,49.0,0.0
54,l2,103.0,102.0,101.0,100.0,99.0,1.0


In [34]:
forecaster = ForecasterRecursiveMultiSeries(
                     regressor          = LinearRegression(),
                     lags               = 5,
                     encoding           = 'ordinal'
                )
forecaster.fit(series=series_2)
results = forecaster.predict(steps=5)
results

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:471                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

,level,pred
50,l1,50.0
50,l2,100.0
51,l1,51.0
51,l2,101.0
52,l1,52.0
52,l2,102.0
53,l1,53.0
53,l2,103.0
54,l1,54.0
54,l2,104.0
